In [27]:
%load_ext autotime

from urllib.request import urlopen
import requests

import bs4
from bs4 import BeautifulSoup as soup

import pandas as pd

import re

import os

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.12 ms (started: 2021-10-10 21:59:10 +07:00)


# Excersise 1

In [28]:
def crawl_page(url):
    Client = urlopen(url)
    xml_page = Client.read()
    Client.close()

    soup_page = soup(xml_page, 'html')

    btn_next = soup_page.find('li', {'class': 'next'})

    while btn_next:
        Client = urlopen(url+btn_next.a['href'])
        xml_page = Client.read()
        Client.close()

        next_page = soup(xml_page, 'html')

        insert_to_soup(soup_page,next_page.findAll('div',{'class':'quote'}))

        btn_next = next_page.find('li', {'class': 'next'})

    export_txt(soup_page)
    print_information(soup_page)


def insert_to_soup(soup_page,div_quote):
    for i in div_quote:
        html = soup_page.findAll('div',{'class':'quote'})
        html[-1].insert_after(i)
    return soup_page

time: 1.32 ms (started: 2021-10-10 21:59:11 +07:00)


In [29]:
def export_txt(source):
    with open('kq.txt', 'w+') as f:
        f.write(str(source))

time: 702 µs (started: 2021-10-10 21:59:12 +07:00)


In [30]:
def print_information(source):
    result = source.findAll('div',{'class':'quote'})
    small = []
    for i in result:
        print(f'Result:\n{i}')
        tmp_small = i.find('small', {'class': 'author'}).text
        small.append(tmp_small)
        print(f'\nSmall:{tmp_small}\n\n')

        tacgiaLink(i,tmp_small)

time: 980 µs (started: 2021-10-10 21:59:12 +07:00)


In [58]:
info_arr = []
def tacgiaLink(result,name):
    quote = result.find('span', {'class': 'text'}).text
    quote = re.sub(r'[“”]','"',quote)
    quote = re.sub(r'′','\'',quote)
    link = result.find('a')['href']

    url = 'https://quotes.toscrape.com'+link

    response = requests.get(url)
    author_page = soup(response.content, 'html')

    birth = author_page.find('span', {'class':'author-born-date'}).text

    info_arr.append([name,link,birth,quote])

time: 1.17 ms (started: 2021-10-10 22:38:16 +07:00)


In [59]:
crawl_page('https://quotes.toscrape.com/')

Result:
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

Small:Albert Einstein


Result:
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <smal

In [60]:
df = pd.DataFrame(info_arr,columns=['Tacgia','Link','Namsinh','Quote'])
df.head()

Tacgia                     Link            Namsinh  \
0  Albert Einstein  /author/Albert-Einstein     March 14, 1879   
1     J.K. Rowling      /author/J-K-Rowling      July 31, 1965   
2  Albert Einstein  /author/Albert-Einstein     March 14, 1879   
3      Jane Austen      /author/Jane-Austen  December 16, 1775   
4   Marilyn Monroe   /author/Marilyn-Monroe      June 01, 1926   

                                               Quote  
0  "The world as we have created it is a process ...  
1  "It is our choices, Harry, that show what we t...  
2  "There are only two ways to live your life. On...  
3  "The person, be it gentleman or lady, who has ...  
4  "Imperfection is beauty, madness is genius and...

time: 6.9 ms (started: 2021-10-10 22:40:48 +07:00)


In [61]:
df = df.groupby(['Tacgia','Link','Namsinh'])['Quote'].apply(list).to_frame().reset_index()
df.head()

Tacgia                          Link            Namsinh  \
0       Albert Einstein       /author/Albert-Einstein     March 14, 1879   
1  Alexandre Dumas fils  /author/Alexandre-Dumas-fils      July 27, 1824   
2       Alfred Tennyson       /author/Alfred-Tennyson    August 06, 1809   
3        Allen Saunders        /author/Allen-Saunders     April 24, 1899   
4            André Gide            /author/Andre-Gide  November 22, 1869   

                                               Quote  
0  ["The world as we have created it is a process...  
1  ["The difference between genius and stupidity ...  
2  ["If I had a flower for every time I thought o...  
3  ["Life is what happens to us while we are maki...  
4  ["It is better to be hated for what you are th...

time: 12.8 ms (started: 2021-10-10 22:40:48 +07:00)


In [62]:
df.to_csv('quotes.csv',index =False)

time: 3.28 ms (started: 2021-10-10 22:40:48 +07:00)
